### Experiment :
### Objectives

1. Train on embeddings and withou embeddinsgs
4. Error analysis 

### Observations

1. 

### Solutions and further investigation

1. 
2. 
3. 

### Load Modeules and datasets

In [1]:
import numpy as np
import random as rn
np.random.seed(0)
rn.seed(0)
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 1000)
# import helper function script
import sys
sys.path.insert(1,'G:\\Github\\Sinhala-Hate-Speech-Detection')
import utills
import TextClassifier


Instructions for updating:
non-resource variables are not supported in the long term


In [71]:


gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.3
rate_drop_dense = 0.3
import tensorflow as tf
from keras.engine import Layer
from tensorflow.keras import backend as K
from  tensorflow.keras.layers import (

    Activation,
    Bidirectional,
    Dense,
    Dropout,
    Embedding,
    Flatten,
    GRU,
    Input,
    SpatialDropout1D,
)
from keras.models import Model
from tensorflow.python.keras.optimizer_v2.adam import Adam

In [70]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale
    
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        print("input_num_capsule =",input_num_capsule )
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]
        print("u-hat ",u_hat_vecs)
        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        print("b = ",b.shape)
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            #outputs = self.activation(tf.matmul(c,u_hat_vecs))
            print("u-hat ",u_hat_vecs)
            print("outputs = ",outputs.shape)
            if i < self.routings - 1:
                print("b = ",b.shape)
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])
               #b = tf.matmul(outputs,u_hat_vecs,transpose_b=True)
                #b = tf.matmul(outputs,u_hat_vecs)


        return outputs #tf.squeeze(outputs)

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [1]:
# uncderstanding matmul vs batch_dot
import tensorflow as tf
from keras.engine import Layer
from tensorflow.keras import backend as K

x_batch = K.ones(shape=(32, 20, 1))
y_batch = K.ones(shape=(32, 30, 20))
xy_batch_dot = K.batch_dot(x_batch, y_batch, axes=[1, 2])
K.int_shape(xy_batch_dot)

(32, 1, 30)

In [74]:
def build_model(TC):
    from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
    inp = Input( shape=(TC.MAX_SEQ_LEN,))
    if(TC.EMBEDDING == None):
        embedding_layer = Embedding(output_dim=TC.EMBEDDING_SIZE, 
                        input_dim=TC.LEN_VOCAB, 
                        input_length=TC.MAX_SEQ_LEN,
                        #weights=[TC.emb_matrix], # Additionally we give the Wi
                        trainable=TC.trainable)(inp)
    else:
        embedding_layer = Embedding(output_dim=TC.EMBEDDING_SIZE, 
                            input_dim=TC.LEN_VOCAB, 
                            input_length=TC.MAX_SEQ_LEN,
                            weights=[TC.emb_matrix], # Additionally we give the Wi
                            trainable=TC.trainable)(inp)
    embedding_layer = SpatialDropout1D(rate_drop_dense)(embedding_layer)
    x = Bidirectional(GRU(gru_len, activation='relu', dropout=dropout_p, recurrent_dropout=dropout_p, return_sequences=True))(
        embedding_layer)
    capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings,share_weights=True)(x)
    

    capsule = Flatten()(capsule)
    capsule = Dropout(dropout_p)(capsule)
    output = Dense(1, activation="sigmoid")(capsule)
    model = Model(inputs=inp, outputs=output)
    optimizer_adam = Adam(learning_rate=TC.lr)
    model.compile(loss='binary_crossentropy',
                    optimizer=optimizer_adam,
                    metrics=['acc'])
    return model

In [51]:
num_routing = 4
num_capsule = 4
dim_capsule = 6

In [ ]:
def get_model():
    input1_pre = Input(shape=(maxlen,))
    embed_layer1_pre = Embedding(max_features,
                            embed_size,
                            input_length=maxlen,
                            weights=[embedding_matrix],
                            trainable=False)(input1_pre)
    embed_layer1_pre = SpatialDropout1D(0.4)(embed_layer1_pre)

    x_pre = Bidirectional(CuDNNGRU(128, return_sequences=True))(embed_layer1_pre)
    capsule_pre1 = PrimaryCap(x_pre,16,1,1)
    capsule_pre2 = PrimaryCap(x_pre,16,1,3)
    capsule_pre3 = PrimaryCap(x_pre,16,1,5)
    toxiccaps_pre1 = CapsuleLayer(num_capsule=6, dim_capsule=16, routings=7, name='toxiccapspre1')(capsule_pre1)
    toxiccaps_pre2 = CapsuleLayer(num_capsule=6, dim_capsule=16, routings=7, name='toxiccapspre2')(capsule_pre2)
    toxiccaps_pre3 = CapsuleLayer(num_capsule=6, dim_capsule=16, routings=7, name='toxiccapspre3')(capsule_pre3)
#     capsule_pre = Capsule(num_capsule=10, dim_capsule=16, routings=5,share_weights=True)(x_pre)
    # output_capsule = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)))(capsule)
    toxiccaps_pre = concatenate([toxiccaps_pre1,toxiccaps_pre2,toxiccaps_pre3])
    capsule_pre = GlobalMaxPooling1D()(toxiccaps_pre)
    capsule_pre = Dropout(0.25)(capsule_pre)

    input1_post = Input(shape=(maxlen,))
    embed_layer1_post = Embedding(max_features,
                            embed_size,
                            input_length=maxlen,
                            weights=[embedding_matrix],
                            trainable=False)(input1_post)
    embed_layer1_post = SpatialDropout1D(0.4)(embed_layer1_post)

    x_post = Bidirectional(CuDNNGRU(128, return_sequences=True))(embed_layer1_post)
    capsule_post1 = PrimaryCap(x_post,16,1,1)
    capsule_post2 = PrimaryCap(x_post,16,1,3)
    capsule_post3 = PrimaryCap(x_post,16,1,5)
    toxiccaps_post1 = CapsuleLayer(num_capsule=6, dim_capsule=16, routings=7, name='toxiccapspost1')(capsule_post1)
    toxiccaps_post2 = CapsuleLayer(num_capsule=6, dim_capsule=16, routings=7, name='toxiccapspost2')(capsule_post2)
    toxiccaps_post3 = CapsuleLayer(num_capsule=6, dim_capsule=16, routings=7, name='toxiccapspost3')(capsule_post3)
#     capsule_post = Capsule(num_capsule=10, dim_capsule=16, routings=5,share_weights=True)(x_post)
    #  output_capsule = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)))(capsule)
    toxiccaps_post = concatenate([toxiccaps_post1,toxiccaps_post2,toxiccaps_post3])
    capsule_post = GlobalMaxPooling1D()(toxiccaps_post)
    capsule_post = Dropout(0.25)(capsule_post)

    concat = concatenate([capsule_pre,capsule_post])
    output = Dense(6, activation='sigmoid')(concat)

    model = Model(inputs=[input1_pre,input1_post], outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=1e-3,decay=0),
        metrics=['accuracy'])
#     model.summary()
    return model


In [18]:
def predict_proba(arr,TC=TC_W):
    import numpy as np
    from tensorflow.python.keras.preprocessing import sequence
    pred=TC.model.predict(sequence.pad_sequences(TC.token.texts_to_sequences(arr),maxlen=TC.MAX_SEQ_LEN))
    returnable=[]
    for i in pred:
        temp=i[0]
        returnable.append(np.array([1-temp,temp]))
    return np.array(returnable)


def error_analysis(TC,model,Y_pred):
    import numpy as np
    from IPython.display import display
    import matplotlib.pyplot as plt
    import eli5
    from eli5.lime import TextExplainer
    import shap
    from lime.lime_text import LimeTextExplainer
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    from tensorflow.python.keras.preprocessing import sequence

    import tensorflow.keras.backend as K
    #token = Tokenizer(num_words=self.LEN_VOCAB)
    #token.fit_on_texts(self.X_tr)
    lime_explainer= LimeTextExplainer(class_names=[0,1])

    te = TextExplainer(random_state=0)
    distrib_samples = TC.X_train[:100]
    #explainer = shap.DeepExplainer(model, distrib_samples)
    # explain the first 25 predictions
    # explaining each prediction requires 2 * background dataset size runs
    # num_explanations = 25 #len(TC_F.X_test)
    # shap_values = explainer.shap_values(TC.X_test[:num_explanations])
    # shap.initjs()
    # num2word = {}
    arr_index=TC.X_te.index
    # for w in TC.word_index.keys():
    #     num2word[TC.word_index[w]] = w
    # x_test_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), TC.X_test[i]))) for i in range(num_explanations)])
    i=0
    
    for s in TC.X_te:
        if(i==21):
            break
        if(TC.Y_test[arr_index[i]] != Y_pred[i]):
            print(s)
            print("Predicted Label : ",TC.result_map(Y_pred[i])," | Turth Label : ",TC.result_map(TC.Y_test[arr_index[i]]))
            te.fit(TC.X_te[arr_index[i]],predict_proba)
            #shap.force_plot(explainer.expected_value[0], shap_values[0][i], x_test_words[i],matplotlib=True)
            display(te.show_prediction(target_names=[0,1]))
            display(lime_explainer.explain_instance(TC.X_te[arr_index[i]],predict_proba).show_in_notebook(text=True))
        i+=1
        print()

In [77]:
def caps_model1(TC):
    # from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
    inp = Input( shape=(TC.MAX_SEQ_LEN,))
    if(TC.EMBEDDING == None):
        embedding_layer = Embedding(output_dim=TC.EMBEDDING_SIZE, 
                        input_dim=TC.LEN_VOCAB, 
                        input_length=TC.MAX_SEQ_LEN,
                        #weights=[TC.emb_matrix], # Additionally we give the Wi
                        trainable=TC.trainable)(inp)
    else:
        embedding_layer = Embedding(output_dim=TC.EMBEDDING_SIZE, 
                            input_dim=TC.LEN_VOCAB, 
                            input_length=TC.MAX_SEQ_LEN,
                            weights=[TC.emb_matrix], # Additionally we give the Wi
                            trainable=TC.trainable)(inp)
    #embedding_layer = SpatialDropout1D(rate_drop_dense)(embedding_layer)
    
    #capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings,share_weights=True)(x)
    # Layer 2: Conv2D layer with `squash` activation, then reshape to 
    # kernel_size should be smaller than window size... maybe half of window size?
    # [None, num_capsule, dim_vector]
    conv1 = layers.Conv1D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(embedding_layer)
    primarycaps = PrimaryCap( conv1, dim_capsule=dim_capsule, 
                            n_channels=1, kernel_size=1,strides=1, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    caps = CapsuleLayer( num_capsule=num_capsule, dim_capsule=dim_capsule, routings=num_routing, name='caps')(primarycaps)


    # Layer 4: This is an auxiliary layer to replace each capsule with its length. 
    # Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    print( "ner_caps", caps.get_shape())
    #out_caps = Length(name='out_caps')(caps)
    #print( "out_caps", out_caps.get_shape())

    capsule = Flatten()(capsule)
    caps = Dropout(dropout_p)(caps)
    output = Dense(1, activation="sigmoid")(caps)
    model = Model(inputs=inp, outputs=output)
    optimizer_adam = Adam(learning_rate=TC.lr)
    model.compile(loss='binary_crossentropy',
                    optimizer=optimizer_adam,
                    metrics=['acc'])
    return model

##### Fasttext trainable

In [5]:
TC_F = TextClassifier.TextClassifier(EMBEDDING='fasttext')

Load data

X train (4526,) Y train (4526,) X test (1940,) Y test (1940,)
Build vocab

Load Embedding model

Create embedding index

found 818830 word vectors
Check coverage
Found embeddings for 87.50% of vocab
Found embeddings for  96.39% of all text
Added 2197 words to embedding with rare words handling of fasttext
Check coverage
Found embeddings for 100.00% of vocab
Found embeddings for  100.00% of all text
dictionary size:  17577
Embedding matrix 



In [22]:
# Fasttext dynamic embedding - tranable = True 
TC_F.EPOCHS =50
TC_F.BATCH_SIZE =16
TC_F.lr = 0.0001
TC_F.tag = " CapsNet trainable fasttext"
TC_F.trainable = True
tcn_tf_model = tcn_model(TC_F)
tcn_tf_model,hist = TC_F.train_model(tcn_tf_model)
Y_pred = TC_F.model_evaluate(tcn_tf_model,hist)
TC_F.save_predictions(Y_pred,TC_F.tag)

NameError: name 'tcn_model' is not defined

##### Fasttext

In [88]:
# increased dropout to 0.2
TC_F.EPOCHS =2
TC_F.BATCH_SIZE =16
TC_F.lr = 0.0001
TC_F.tag = " CapsNet fasttext"
TC_F.trainable = False

#cps_f_model = build_model(TC_F)
cps_f_model = caps_model1(TC_F)
cps_f_model,hist = TC_F.train_model(cps_f_model)
Y_pred = TC_F.model_evaluate(cps_f_model,hist)
TC_F.save_predictions(Y_pred,TC_F.tag)

(?, 92, 6)
squash
92
Tensor("caps_25/zeros:0", shape=(?, 4, 1, 92), dtype=float32)
Tensor("caps_25/zeros_1:0", shape=(?, 4, 1, 92), dtype=float32)
i = 0
b loop= (?, 4, 1, 92)
c shape (?, 4, 1, 92)
input hat -  <unknown>
before squash s <unknown>
before squash <unknown>
squash
 output =  <unknown>
inside if
b = Tensor("caps_25/add_2:0", dtype=float32)
b shape  <unknown>
i = 1
b loop= <unknown>


TypeError: in user code:

    <ipython-input-87-fcdb3617ab99>:149 call  *
        c = tf.nn.softmax(b, axis=1)
    C:\Users\Kavishka\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    C:\Users\Kavishka\anaconda3\lib\site-packages\tensorflow\python\ops\nn_ops.py:3734 softmax_v2
        return _wrap_2d_function(logits, gen_nn_ops.softmax, axis, name)
    C:\Users\Kavishka\anaconda3\lib\site-packages\tensorflow\python\ops\nn_ops.py:3617 _wrap_2d_function
        is_last_dim = (dim == -1) or (dim == shape.ndims - 1)

    TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'


In [ ]:
# class TextCapsule(object):
#     def __init__(self, training,params):
#         self.training = training
#         self.params = params
#         self.embedding_layer = EmbeddingLayer(vocab_size=params['vocab_size'],
#                                                embed_size=params['embedding_size'],
#                                                embedding_type=params['embedding_type'],
#                                                params=params)
#         self.capsule_layer_conv=Capsule_layer_conv(shape=[3,1,64,64],vec_length=self.params['capsule_vec_length'])
#         self.capsule_layer_dense=Capsule_layer_dense(hidden_size=params['n_class'])

#     def build(self, inputs):
#         with tf.name_scope('embed'):
#             embedding_outputs = self.embedding_layer(inputs)
#         if self.training:
#             embedding_outputs = tf.nn.dropout(embedding_outputs, self.params['embedding_dropout_keep'])

#         embedding_outputs=tf.expand_dims(embedding_outputs,-1)
#         with tf.name_scope('conv'):
#             conv1 = tf.layers.conv2d(inputs=embedding_outputs,
#                                      filters=self.params['filters'],
#                                      kernel_size=[3,300],
#                                      strides=1,
#                                      padding='valid',
#                                      activation=tf.nn.relu) #[batch_size,seq_length-2,1,128]

#         cap_conv=self.capsule_layer_conv(conv1)
#         cap_conv_shape=cap_conv.get_shape().as_list()
#         #cap_flat shape [batch_size, (seq_length-4)* cap_kernel_size[-1]), vec_length]
#         cap_flat=tf.reshape(cap_conv,
#                             [-1,cap_conv_shape[1]*cap_conv_shape[2]*cap_conv_shape[3],self.capsule_layer_conv.vec_length])
#         logits=self.capsule_layer_dense(cap_flat)
#         return logits

#     def __call__(self, inputs, targets=None):
#         logits=self.build(inputs)
#         return logits
def caps_model2(TC):
    # from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
    inp = Input( shape=(TC.MAX_SEQ_LEN,))
    if(TC.EMBEDDING == None):
        embedding_layer = Embedding(output_dim=TC.EMBEDDING_SIZE, 
                        input_dim=TC.LEN_VOCAB, 
                        input_length=TC.MAX_SEQ_LEN,
                        #weights=[TC.emb_matrix], # Additionally we give the Wi
                        trainable=TC.trainable)(inp)
    else:
        embedding_layer = Embedding(output_dim=TC.EMBEDDING_SIZE, 
                            input_dim=TC.LEN_VOCAB, 
                            input_length=TC.MAX_SEQ_LEN,
                            weights=[TC.emb_matrix], # Additionally we give the Wi
                            trainable=TC.trainable)(inp)
    #embedding_layer = SpatialDropout1D(rate_drop_dense)(embedding_layer)
    
    #capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings,share_weights=True)(x)
    # Layer 2: Conv2D layer with `squash` activation, then reshape to 
    # kernel_size should be smaller than window size... maybe half of window size?
    # [None, num_capsule, dim_vector]
    conv1 = layers.Conv1D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(embedding_layer)
    primarycaps = PrimaryCap( conv1, dim_capsule=dim_capsule, 
                            n_channels=1, kernel_size=1,strides=1, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    caps = CapsuleLayer( num_capsule=num_capsule, dim_capsule=dim_capsule, routings=num_routing, name='caps')(primarycaps)


    # Layer 4: This is an auxiliary layer to replace each capsule with its length. 
    # Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    print( "ner_caps", caps.get_shape())
    #out_caps = Length(name='out_caps')(caps)
    #print( "out_caps", out_caps.get_shape())

    capsule = Flatten()(capsule)
    caps = Dropout(dropout_p)(caps)
    output = Dense(1, activation="sigmoid")(caps)
    model = Model(inputs=inp, outputs=output)
    optimizer_adam = Adam(learning_rate=TC.lr)
    model.compile(loss='binary_crossentropy',
                    optimizer=optimizer_adam,
                    metrics=['acc'])
    return model

class Capsule_layer(tf.layers.Layer):
    def __init__(self):
        super(Capsule_layer,self).__init__()

    def squash(self,vector,axis=-1,epsilon=1e-7):
        vec_squared_norm=tf.reduce_sum(tf.square(vector),axis=axis,keepdims=True)
        scale=vec_squared_norm/(1+vec_squared_norm)/tf.sqrt(vec_squared_norm+epsilon)
        return scale*vector

    def dynamic_routing(self,input,iters):
        # input shape [batch_size,n_class, (seq_length-4)* cap_kernel_size[-1]), vec_length]
        # output shape [batch_size, n_class, vec_length]
        b=tf.zeros_like(input[:,:,:,-1], dtype=tf.float32, name='b')
        for i in range(iters):
            c=tf.nn.softmax(b,1)
            output=self.squash(tf.einsum('bij,bijk->bik',c,input))
            if i<iters-1:
                b=b+tf.einsum('bik,bijk->bij',output,input)
        return output

    def vec_transform_conv(self,inputs,input_cap_dim,input_cap_num,output_cap_dim,output_cap_num):
        #output u_hat_vecs shape:[batch_size, hidden_size,(seq_length-4)* cap_kernel_size[-1]),vec_length]
        kernel_size=[1]
        u_hat_vecs=tf.layers.conv1d(inputs=inputs,kernel_size=kernel_size,filters=output_cap_dim*output_cap_num)
        u_hat_vecs=tf.reshape(u_hat_vecs,[-1,input_cap_num,output_cap_num,output_cap_dim])
        u_hat_vecs=tf.transpose(u_hat_vecs,(0,2,1,3))
        return u_hat_vecs


class Capsule_layer_conv(Capsule_layer):
    def __init__(self,shape,vec_length):
        super(Capsule_layer_conv,self).__init__()
        self.shape=shape
        self.vec_length=vec_length

    def call(self, inputs, **kwargs):
        #output cap shape #[batch_size,seq_length-4,1,shape[-1], vec_length]
        cap=tf.layers.conv2d(inputs,filters=self.shape[-1]*self.vec_length,kernel_size=self.shape[:2],strides=1)
        cap_shape=cap.get_shape().as_list()
        cap=tf.reshape(cap,[-1,cap_shape[1],cap_shape[2],self.shape[-1],self.vec_length])
        cap=self.squash(cap)
        return cap


class Capsule_layer_dense(Capsule_layer):
    def __init__(self,hidden_size):
        super(Capsule_layer_dense,self).__init__()
        self.hidden_size=hidden_size

    def call(self, inputs, **kwargs):
        inputs_shape=inputs.get_shape().as_list()
        cap=self.vec_transform_conv(inputs,input_cap_dim=inputs_shape[-1],input_cap_num=inputs_shape[1],
                                    output_cap_dim=inputs_shape[-1],output_cap_num=self.hidden_size)
        outputs=self.dynamic_routing(cap,iters=3)
        #outputs shape [batch_size, n_class]
        outputs=tf.sqrt(tf.reduce_sum(tf.square(outputs),axis=2))
        return outputs

In [87]:
"""
Some key layers used for constructing a Capsule Network. These layers can used to construct CapsNet on other dataset, 
not just on MNIST.
*NOTE*: some functions can be implemented in multiple ways, I keep all of them. You can try them for yourself just by
uncommenting them and commenting their counterparts.
Author: Xifeng Guo, E-mail: `guoxifeng1990@163.com`, Github: `https://github.com/XifengGuo/CapsNet-Keras`
"""

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import initializers, layers


class Length(layers.Layer):  # CapsuleNorm
    """
    Compute the length of vectors. This is used to compute a Tensor that has the same shape with y_true in margin_loss.
    Using this layer as model's output can directly predict labels by using `y_pred = np.argmax(model.predict(x), 1)`
    inputs: shape=[None, num_vectors, dim_vector]
    output: shape=[None, num_vectors]
    """
    def call(self, inputs, **kwargs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), -1) + K.epsilon())

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

    def get_config(self):
        config = super(Length, self).get_config()
        return config


class Mask(layers.Layer):
    """
    Mask a Tensor with shape=[None, num_capsule, dim_vector] either by the capsule with max length or by an additional 
    input mask. Except the max-length capsule (or specified capsule), all vectors are masked to zeros. Then flatten the
    masked Tensor.
    For example:
        ```
        x = keras.layers.Input(shape=[8, 3, 2])  # batch_size=8, each sample contains 3 capsules with dim_vector=2
        y = keras.layers.Input(shape=[8, 3])  # True labels. 8 samples, 3 classes, one-hot coding.
        out = Mask()(x)  # out.shape=[8, 6]
        # or
        out2 = Mask()([x, y])  # out2.shape=[8,6]. Masked with true labels y. Of course y can also be manipulated.
        ```
    """
    def call(self, inputs, **kwargs):
        if type(inputs) is list:  # true label is provided with shape = [None, n_classes], i.e. one-hot code.
            assert len(inputs) == 2
            inputs, mask = inputs
        else:  # if no true label, mask by the max length of capsules. Mainly used for prediction
            # compute lengths of capsules
            x = tf.sqrt(tf.reduce_sum(tf.square(inputs), -1))
            # generate the mask which is a one-hot code.
            # mask.shape=[None, n_classes]=[None, num_capsule]
            mask = tf.one_hot(indices=tf.argmax(x, 1), depth=x.shape[1])

        # inputs.shape=[None, num_capsule, dim_capsule]
        # mask.shape=[None, num_capsule]
        # masked.shape=[None, num_capsule * dim_capsule]
        masked = K.batch_flatten(inputs * tf.expand_dims(mask, -1))
        return masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  # true label provided
            return tuple([None, input_shape[0][1] * input_shape[0][2]])
        else:  # no true label provided
            return tuple([None, input_shape[1] * input_shape[2]])

    def get_config(self):
        config = super(Mask, self).get_config()
        return config


def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param vectors: some vectors to be squashed, N-dim tensor
    :param axis: the axis to squash
    :return: a Tensor with same shape as input vectors
    """
    print("squash")
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors


class CapsuleLayer(layers.Layer):
    """
    The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_capsule] and output shape = \
    [None, num_capsule, dim_capsule]. For Dense Layer, input_dim_capsule = dim_capsule = 1.
    :param num_capsule: number of capsules in this layer
    :param dim_capsule: dimension of the output vectors of the capsules in this layer
    :param routings: number of iterations for the routing algorithm
    """
    def __init__(self, num_capsule, dim_capsule, routings=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]
        print(self.input_num_capsule)
        # Transform matrix, from each input capsule to each output capsule, there's a unique weight as in Dense layer.
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True

    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsule, input_dim_capsule]
        # inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule, 1]
        inputs_expand = tf.expand_dims(tf.expand_dims(inputs, 1), -1)

        # Replicate num_capsule dimension to prepare being multiplied by W
        # inputs_tiled.shape=[None, num_capsule, input_num_capsule, input_dim_capsule, 1]
        inputs_tiled = tf.tile(inputs_expand, [1, self.num_capsule, 1, 1, 1])

        # Compute `inputs * W` by scanning inputs_tiled on dimension 0.
        # W.shape=[num_capsule, input_num_capsule, dim_capsule, input_dim_capsule]
        # x.shape=[num_capsule, input_num_capsule, input_dim_capsule, 1]
        # Regard the first two dimensions as `batch` dimension, then
        # matmul(W, x): [..., dim_capsule, input_dim_capsule] x [..., input_dim_capsule, 1] -> [..., dim_capsule, 1].
        # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsule]
        inputs_hat = tf.squeeze(tf.map_fn(lambda x: tf.matmul(self.W, x), elems=inputs_tiled))

        # Begin: Routing algorithm ---------------------------------------------------------------------#
        # The prior for coupling coefficient, initialized as zeros.
        # b.shape = [None, self.num_capsule, 1, self.input_num_capsule].
        
        print(tf.zeros(shape=[K.shape(inputs)[0], self.num_capsule, 1, self.input_num_capsule]))
        b = tf.zeros(shape=[K.shape(inputs)[0], self.num_capsule, 1, self.input_num_capsule])
    
        print(b)
        assert self.routings > 0, 'The routings should be > 0.'
        for i in range(self.routings):
            print("i =",i)
            # c.shape=[batch_size, num_capsule, 1, input_num_capsule]
            print('b loop=',b.get_shape())
            c = tf.nn.softmax(b, axis=1)
            print("c shape",c.shape)
            print('input hat - ',inputs_hat.shape)
            # c.shape = [batch_size, num_capsule, 1, input_num_capsule]
            # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
            # The first two dimensions as `batch` dimension,
            # then matmal: [..., 1, input_num_capsule] x [..., input_num_capsule, dim_capsule] -> [..., 1, dim_capsule].
            # outputs.shape=[None, num_capsule, 1, dim_capsule]
            #s = tf.einsum('bij,bijk->bik',c,inputs_hat)
            s = tf.matmul(c, inputs_hat)
            print("before squash s",s.shape)
            print("before squash",s.shape)
            outputs = squash(s)  # [None, 10, 1, 16]
            print(" output = ",outputs.shape)
            if i < self.routings - 1:
                # outputs.shape =  [None, num_capsule, 1, dim_capsule]
                # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
                # The first two dimensions as `batch` dimension, then
                # matmal:[..., 1, dim_capsule] x [..., input_num_capsule, dim_capsule]^T -> [..., 1, input_num_capsule].
                # b.shape=[batch_size, num_capsule, 1, input_num_capsule]
                print("inside if")
                b += tf.matmul(outputs, inputs_hat, transpose_b=True)
                print("b =",b)
                print("b shape ",b.shape)
        # End: Routing algorithm -----------------------------------------------------------------------#

        return tf.squeeze(outputs)

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])

    def get_config(self):
        config = {
            'num_capsule': self.num_capsule,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
        }
        base_config = super(CapsuleLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    """
    Apply Conv2D `n_channels` times and concatenate all capsules
    :param inputs: 4D tensor, shape=[None, width, height, channels]
    :param dim_capsule: the dim of the output vector of capsule
    :param n_channels: the number of types of capsules
    :return: output tensor, shape=[None, num_capsule, dim_capsule]
    """
    output = layers.Conv1D(filters=dim_capsule*n_channels, kernel_size=kernel_size, strides=strides, padding=padding,
                           name='primarycap_conv2d')(inputs)
    #output = layers.Conv1D(filters=dim_capsule*n_channels, kernel_size=kernel_size,name='primarycap_conv1d')(inputs)    
    print(output.shape)                  
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)
   # return layers.Lambda(squash)(output)

"""
# The following is another way to implement primary capsule layer. This is much slower.
# Apply Conv2D `n_channels` times and concatenate all capsules
def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    outputs = []
    for _ in range(n_channels):
        output = layers.Conv2D(filters=dim_capsule, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)
        outputs.append(layers.Reshape([output.get_shape().as_list()[1] ** 2, dim_capsule])(output))
    outputs = layers.Concatenate(axis=1)(outputs)
    return layers.Lambda(squash)(outputs)
"""

'\n# The following is another way to implement primary capsule layer. This is much slower.\n# Apply Conv2D `n_channels` times and concatenate all capsules\ndef PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):\n    outputs = []\n    for _ in range(n_channels):\n        output = layers.Conv2D(filters=dim_capsule, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)\n        outputs.append(layers.Reshape([output.get_shape().as_list()[1] ** 2, dim_capsule])(output))\n    outputs = layers.Concatenate(axis=1)(outputs)\n    return layers.Lambda(squash)(outputs)\n'

In [ ]:
error_analysis(TC_F,tcn_f_model,Y_pred)

##### word2vec skipgram

In [ ]:
TC_W = TextClassifier.TextClassifier(EMBEDDING='w2v_skipgram')

In [ ]:
# increase dropout to 0.2
TC_W.EPOCHS =50
TC_W.BATCH_SIZE =16
TC_W.lr = 0.0001
TC_W.tag = "CapsNet w2v skipgram "
tcn_w_model = build_model(TC_W)
tcn_w_model, hist = TC_W.train_model(tcn_w_model)
Y_pred = TC_W.model_evaluate(tcn_w_model,hist)
TC_W.save_predictions(Y_pred,TC_W.tag)

In [ ]:
error_analysis(TC_W,tcn_w_model,Y_pred)

##### word2vec cbow

In [ ]:
TC_W_2 = TextClassifier.TextClassifier(EMBEDDING='w2v_cbow')

In [ ]:
# dropout 0.2
TC_W_2.EPOCHS =50
TC_W_2.BATCH_SIZE =16
TC_W_2.lr = 0.0001
TC_W_2.tag = "CapsNet w2v cbow"
tcn_w2_model = build_model(TC_W_2)

tcn_w2_model, hist = TC_W_2.train_model(tcn_w2_model)
Y_pred = TC_W_2.model_evaluate(tcn_w2_model,hist)
TC_W_2.save_predictions(Y_pred,TC_W_2.tag)

In [ ]:
error_analysis(TC_W_2,tcn_w2_model,Y_pred)

##### Random embedding

In [3]:
TC = TextClassifier.TextClassifier(EMBEDDING=None)

Load data

X train (4526,) Y train (4526,) X test (1940,) Y test (1940,)
dictionary size:  17577


In [ ]:
# Fasttext dynamic embedding - tranable = True - changesd architecture
TC.EPOCHS =50
TC.BATCH_SIZE =16
TC.lr = 0.0001
TC.tag = "CapsNet random emb"
TC.trainable = True
tcn_tf_model = build_model(TC)
tcn_tf_model,hist = TC.train_model(tcn_tf_model)
Y_pred = TC.model_evaluate(tcn_tf_model,hist)
TC.save_predictions(Y_pred,TC.tag)